# 1. Getting Started with PyLapi

Python Lightweight API (PyLapi) is a Python API builder. It takes only a few seconds to automatically generate a Python API from OpenAPI specifications or less than an hour for an experienced Python developer to create a custom Python API.

You can drive PyLapi in an [automatic](./2.%20How%20to%20use%20a%20PyLapi%20API.ipynb), [semiautomatic](./5.%20PyLapi%20Generator%20Automation.ipynb), or [manual](./3.%20A%20ChatGPT%20Conversation%20with%20PyLapi.ipynb) way.

***Automatic***:

The auto-generated Python API contains fully functional resource classes with API methods embedded with OpenAPI specifications as user guide inline comments, so you don't need to read the lengthy OpenAPI Specification to study or use the API.

For more details, please see the [Automatic API Generation](#automatic-api-generation) section and this [tutorial](./2.%20How%20to%20use%20a%20PyLapi%20API.ipynb) on how to use the generated API.

***Semiautomatic***:

You can also manually "rewrite" some of the generated classes and methods in a separate code file to, for example, specify custom parameters, pre-process the request, or post-process the response. The PyLapi generator will merge your code into the generated Python API automatically.

PyLapi's code-rewrite feature allows you to specify different API URL endpoints for individual resources, effectively combining multiple API backend services into a single frontend API. All parameters and the request body are dynamically mapped based on the API method route, with the option to rewrite them if you so choose.

More details can be found in this [tutorial](./5.%20PyLapi%20Generator%20Automation.ipynb).

***Manual***:

When OpenAPI Specifications are not available or you want to create a Python API your own way, you can inherit your root API class from PyLapi, which does all the heavy lifting for you, so your code would mostly be a `pass`.

If you want to manipulate the API requests and responses, only minimal coding is required. In the tutorial [A ChatGPT Conversation with PyLapi](./3.%20A%20ChatGPT%20Conversation%20with%20PyLapi.ipynb), a fully functional Conversation resource is completed in 38 lines, allowing you to do cool things like this:

```python
conversation.ask("Where is the Sydney Opera House?")
print(conversation.data.answers[0])
# Output:
# The Sydney Opera House is located in Sydney, Australia. Specifically, it is situated on Bennelong Point in the Sydney Harbour, close to the Sydney Harbour Bridge.
```

---
Here is what a PyLapi-supported OpenAI (ChatGPT) API would look like:
<br/><br/>

```python
from pylapi import PyLapi

class oAPI(PyLapi):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.api_url = "https://api.openai.com/v1"

@oAPI.resource_class(resource_name="conversation")
class Conversation(oAPI):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.message_bank = []

    @oAPI.resource_method(method_path="chat/completions", http_method="POST", load="$")
    def ask(self, question: str):

        @oAPI.callback
        def request(self, role="user", model="gpt-3.5-turbo", **kwargs):
            self.message_bank.append({
                "role": role,
                "content": kwargs["question"],
            })
            self.raw_request["json"] = {
                "model": model,
                "messages": self.message_bank,
                "temperature": 0.7
            }
            self.raw_request["params"] = {}

        @oAPI.callback
        def response(self, **kwargs):
            if self.response_ok() and "choices" in self.response_data:
                choices = self.response_data["choices"]
                self.message_bank.append(choices[0]["message"])
                answers = [_["message"]["content"] for _ in choices]
                self.response_data.update({
                    "answers": answers,
                })
```

<br/><br/>
The `oAPI` root API class inherits the decorators `@oAPI.resource_class` and `@oAPI.resource_methods` from `PyLapi`, which prepares the API parameters and the request body in the API request, sends the request to the API URL endpoint, processes the response, and handles the resource data for you.

---
Using a PyLapi-supported Python API is easy and natural.
You may use keyword arguments, positional arguments, or implicit arguments from the `data` attribute.

For example,
<br/><br/>

```python
oAPI.auth(<API_KEY>)
conversation = oAPI.resource("conversation")
conversation.ask("Where is the Sydney Opera House?")
print(conversation.data.answers[0])
```

## Install PyLapi

Please follow these instructions to install PyLapi and its generator (replacing all environment variables with your specific values).

To install the PyLapi class:
```bash
pip install pylapi
```

To install the PyLapi generator:

```bash
cd $PATH_TO_DEV_ENV
git clone https://github.com/jackyko8/pylapi.git
cd pylapi
cp tools/pylapi_gen* $PATH_TO_BIN
```

## Automatic API Generation

To generate a PyLapi supported Python API:

```bash
cd $PATH_TO_MYAPI
cp $PATH_TO_DEV_ENV/pylapi/tools/pylapi_config_template.py myapi_config.py
# Configure myapi_config.py
pylapi_gen myapi_config.py
# Output
# MyAPI generated and saved in ./myapi.py
```

In this tutorial, we use the [GitHub REST API](https://docs.github.com/en/rest) as an example to illustrate how to automatically generate a Python API using PyLapi.

The OpenAPI Specification (OAS) of the GitHub v3 REST API can be found [online](https://github.com/github/rest-api-description/tree/main/descriptions/api.github.com).
For demonstration purposes, we have downloaded the OAS JSON document [here](./api.github.com.json).

### Configuration

We have prepared a PyLapi configuration file for the GitHub REST API in [./gapi_config.py](./gapi_config.py).

In [1]:
!cat ./gapi_config.py

# gAPI (GitHub API) PyLapi Generator Configuration

api_class_name = "gAPI"
output_py_name = "./gapi.py"
oas_file_name = "./api.github.com.json"

guide_attrs = {
    "summary",
    "description",
    "parameters",
    "request_body",
    # "ref",
}

naming = {
    "class_name": "upperCamel($.path_segments[0]) + 'Resource'",
    "resource_name": "snake($.path_segments[0])",
    "class_path": "$.path_segments[0]",
    "resource_path": "'/'.join($.path_segments[1:])",
    "method_name": "lowerCamel($.operation_id)",
}


In the above configuration example, we are generating an API class called `gAPI` and storing the Python script in [./gapi.py](./gapi.py).
The OAS (OpenAPI Specification) is read from [./api.github.com.json](./api.github.com.json).

We also include some OAS attributes as a user guide embedded in the Python script as inline comments.

For `naming`, let us use the "Get repository content" method to illustrate. It is the method at line 31692 in the [OAS file](./api.github.com.json), or you may search for the API path `/repos/{owner}/{repo}/contents/{path}` or the operationId `repos/get-content`.

| Item          | Conversion                                  | Value                          |
| ------------- | ------------------------------------------- | ------------------------------ |
| class_name    | upperCamel($.path_segments[0]) + 'Resource' | ReposResource                  |
| resource_name | snake($.path_segments[0])                   | repos                          |
| class_path    | $.path_segments[0]                          | repos                          |
| resource_path | '/'.join($.path_segments[1:])               | {owner}/{repo}/contents/{path} |
| method_name   | lowerCamel($.operation_id)                  | reposGetContent                |

Notes:
1. The `$.` prefix indicates a method attribute, which we will discuss in later tutorials.
2. The naming specifications are processed by `eval`, so if you want to specify an empty string, use `"''"`. For example, `"resource_name": "''"`.

### Generation

You may now run `pylapi_gen` to generate the `gAPI` script based on the configuration in `./gapi.py`:

```bash
pylapi_gen gapi_config.py
```

(The `../tools/` relative path is not required if you already have `pylapi_gen` in your execution path.)

In [2]:
!../tools/pylapi_gen gapi_config.py

gAPI generated and saved in ./gapi.py


Voilà! The `gAPI` Python API has been generated for you in seconds.
You may now open [./gapi.py](./gapi.py) and search for `ReposResource` and `reposGetContent` to verify if the naming is right.

---
In the next tutorial, we are going to show you [how to use a PyLapi API](2.%20How%20to%20use%20a%20PyLapi%20API.ipynb) using this newly generated `gAPI` as an example.

## End of page